In [4]:
import json
from osgeo import gdal
import pdal
import os
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pprint import pprint
import laspy
import time
import sys

gdal.UseExceptions()

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()

from utils import create_all_folders, Folders

create_all_folders()

In [2]:
# Define tile size and OVERLAP
# TILE_SIZE = 1920  # Size of each tile
# OVERLAP = 480  # Overlap between tiles
TILE_SIZE = 640  # Size of each tile
OVERLAP = 0  # Overlap between tiles

In [3]:
from utils import create_folder, get_file_base_name, open_json
from data_processing import (
    get_image_path_from_full_annotation_path,
    get_cropping_limits,
)

### Get the cropping limits

In [10]:
from data_processing import (
    find_annots_repartition,
    crop_annots_into_limits,
    annots_coordinates_to_local,
    save_annots_per_image,
    crop_all_images_from_annotations_folder,
)

In [12]:
annotations_path = Folders.FULL_ANNOTS.value + "3"

from chm import compute_chm


# Put everything below in a function and add also the creation of the full point cloud
# def preprocess_data(annotations_path: str) -> None:

annotations = open_json(annotations_path)

full_image_path_tif = get_image_path_from_full_annotation_path(annotations)

cropping_limits_x, cropping_limits_y = get_cropping_limits(
    full_image_path_tif, TILE_SIZE, OVERLAP
)
visibility_threshold = 0.2
annots_repartition = find_annots_repartition(
    cropping_limits_x, cropping_limits_y, annotations, visibility_threshold
)
crop_annots_into_limits(annots_repartition)
annots_coordinates_to_local(annots_repartition)

output_image_prefix = get_file_base_name(full_image_path_tif)
annotations_output_directory = os.path.join(
    Folders.CROPPED_ANNOTS.value, output_image_prefix
)
save_annots_per_image(
    annots_repartition, annotations_output_directory, full_image_path_tif
)

images_output_directory = os.path.join(
    Folders.CROPPED_IMAGES.value, output_image_prefix
)
crop_all_images_from_annotations_folder(
    annotations_output_directory, images_output_directory
)